In [1]:
import pandas as pd


original = pd.read_csv('data.csv')
original = original.drop(['都道府県名ローマ字', '市区町村名ローマ字', '大字町丁目コード', '大字町丁目名', '緯度', '経度'], axis=1)
original = original.rename(columns={'都道府県コード':  'prefecture_code', 
                                                                 '都道府県名': ' prefecture_name',
                                                                 '都道府県名カナ': 'prefecture_name_ruby', 
                                                                 '市区町村コード': 'city_code',
                                                                 '市区町村名': 'city_name',
                                                                 '市区町村名カナ': 'city_name_ruby',})

station = pd.read_csv('station.csv')
station = station.drop(['station_name_k', 'station_name_r', 'station_g_cd', 'line_cd', 'post', 'lon', 'lat', 'open_ymd', 'close_ymd', 'e_status', 'e_sort'],axis=1)
station = station.rename(columns={'station_cd': 'station_code', 'pref_cd': 'prefecture_code'})



prefecture = original.iloc[:,:3]
counter_prefecture = prefecture.iloc[0, 0]
drop_prefecture = []
for i in range(len(prefecture) - 1):
    if prefecture.iloc[i, 0] == counter_prefecture:
        drop_prefecture.append(i)
    else:
        counter_prefecture = prefecture.iloc[i, 0]
        continue
prefecture = prefecture.drop(drop_prefecture)

city = original.drop(columns=original.columns[[1, 2]])
counter_city = city.iloc[0,1]
drop_city = []
for i in range(1, len(city) - 1):
    if city.iloc[i, 1] == counter_city:
        drop_city.append(i)
    else:
        counter_city = city.iloc[i, 1]
        continue
city =  city.drop(drop_city)

print(station)



append_city_code_list = []
for i in range(len(station) - 1):
    append_code = 0
    for j in range(len(city) - 1):
        if city.iloc[j, 2] in station.iloc[i, 3]:
            # append_city_code_list.append(city.iloc[j, 1])
            append_code = city.iloc[j, 1]
    append_city_code_list.append

print(len(append_city_code_list))

       station_code station_name  prefecture_code           address
0           1110101           函館                1    北海道函館市若松町１２-１３
1           1110102          五稜郭                1           函館市亀田本町
2           1110103           桔梗                1  北海道函館市桔梗３丁目４１-３６
3           1110104          大中山                1       亀田郡七飯町大字大中山
4           1110105           七飯                1         亀田郡七飯町字本町
...             ...          ...              ...               ...
10850       9992719        てだこ浦西               47     沖縄県浦添市前田三丁目21
10851       9992801      九州鉄道記念館               40        福岡県北九州市門司区
10852       9992802        出光美術館               40        福岡県北九州市門司区
10853       9992803     ノーフォーク広場               40     福岡県北九州市門司区旧門司
10854       9992804      関門海峡めかり               40      福岡県北九州市門司区門司

[10855 rows x 4 columns]
        prefecture_code  city_code city_name city_name_ruby
0                     1       1101    札幌市中央区    サッポロシチュウオウク
990                   1       1102    